In [1]:
import pandas as pd
from get_gene_info import get_genome_info, get_genes_from_ranges

Getting the genes that should be affected by the deleted ranges (Only for Delta 16 from MG1655 and all the W3110 strains as the others already report which genes are deleted)


In [2]:
# MG1655

delta16 = pd.read_csv('../files/deleted_ranges/delta_16.csv',encoding='latin1')

genomeMG1655_path = '../files/genomes/MG1655.gb'
genomeMG1655 = get_genome_info(genomeMG1655_path)

delta_16_genes = get_genes_from_ranges(delta16, genomeMG1655)

delta_16_genes.to_csv('../files/deleted_genes/mapped_del_genes_Δ16.csv',header=False)

1170


In [3]:
# W3110 
rangos = pd.read_csv('../files/deleted_ranges/MGF_line.csv',encoding='latin1')
genomeW3110_path = '../files/genomes/W3110.gb'
genomeW3110 = get_genome_info(genomeW3110_path)

MGF01 = rangos.loc[:,['MGF01_L','MGF01_R']].dropna()
MGF02 = rangos.loc[:,['MGF02_L','MGF02_R']].dropna()
DGF298 = rangos.loc[:,['DGF298_L','DGF298_R']].dropna()
DGF327 = rangos.loc[:,['DGF327_L','DGF327_R']].dropna()

strains = [MGF01,MGF02,DGF298,DGF327]

genes_finales = {}
for strain in strains:
    nombre = strain.columns[0].split('_')[0]
    genes_finales[nombre] = get_genes_from_ranges(strain, genomeW3110)


for strain in genes_finales.keys():
    genes_finales[strain].to_csv('../files/deleted_genes/mapped_del_genes_'+strain+'.csv',header=False)

1035
1176
1727
1392
